# Bike Montreal data science (Daily user predicton)

**Diclaimer**: If anyone is reading this, please note that this project is my first data science project. It is my first time using Jupyter notebooks, pandas, SQL and many other things. I have also deliberately decided to not follow a similar project as an example because I feared I would simply copy the work and not learn as much in consequence.  This means that this project will **not** follow best practices in the industry or even correct usage of tools and libraries. My goal was to blindly jump into a project and learn as I go.

## Objective

[Bixi](https://bixi.com/en) is Montreal's public bike share program. It has been in service since 2014 and now has close to 800 stations with close to 10000 bikes in the network. 

In a previous project where I first learned Python and webscrapping, I scrapped Bixi's open data since 2014 and the historical daily weather of Montreal. 

In this project, I will attempt to predict the daily usage of Bixi bikes based on the different trends and weather condition.

## Sections (TODO)

## Creating our dataframe

Let's query from **gbq** daily usage data and daily weather and save it insida a dataframe. Let's do this for everyday since the beginning of Bixi's history.

In [1]:
from helpers import get_min_value

In [2]:
# TODO find min day in bixi dataset
# TODO find max day in bixi dataset
# create empty dataframe
# Loop through each day and append date, daily bixi count, all weather variables.